In [25]:
import os
import glob
import mne
import csv
import matplotlib.pyplot as plt
import numpy as np
import scipy
from fooof import FOOOF, FOOOFGroup, fit_fooof_3d
from fooof.sim.gen import gen_aperiodic
import json

In [26]:
def compute_psds(age):
    subjects = [os.path.split(x)[-1] for x in glob.glob('/home/common/bonaiuto/devmobeta/data/sub-*')]
    subject_age_path = os.path.join('/home/common/bonaiuto/devmobeta/derivatives/')
    subject_age = os.path.join(subject_age_path, 'participants_v2.csv')

    sub_psds = []
    freqs = None
            
    with open(subject_age, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print("Processing row:", row)
            subject = row['subject_id']
            session = row['session']
            subject_age = row['age']
            system = row['eeg_system']

            if system != "BrainVision":
                print(f"Skipping subject {subject} (EEG system: {system})")
                continue 
                            
            if subject_age == age:
                print(f"Processing subject {subject} with age {subject_age} (BrainVision)")
                sub_path = os.path.join('/home/common/bonaiuto/devmobeta/derivatives/', subject)
                ses_path = os.path.join(sub_path, session)
                print(ses_path)
                
                if os.path.exists(os.path.join(ses_path, 'eeg', 'NEARICA_NF')): 
                    print(f"Processing subject {subject} with age {subject_age} and session {session}")
                    eeg_path = os.path.join(ses_path, 'eeg', 'NEARICA_NF')
                    fname = os.path.join(eeg_path, f'03_ica_data/{subject}_task-devmobeta_grasp_eeg_ica_art_rej_interp_reref.set')

                    if os.path.exists(fname):
                        raw_data = mne.io.read_raw_eeglab(fname, preload=True)
                        srate = raw_data.info['sfreq']

                        # Prendre tous les canaux EEG disponibles
                        all_chan_idx = np.arange(len(raw_data.info['ch_names']))

                        # Extraire les données des canaux EEG
                        data = raw_data.get_data()[all_chan_idx, :]
                        n_chans, n_samps = data.shape

                        # Utiliser une fenêtre de 1s avec un chevauchement de 50%
                        winsize = srate
                        overlap = int(winsize / 2)    

                        # Calcul de la PSD pour tous les canaux
                        freqs, chan_psd = scipy.signal.welch(
                            data, 
                            fs=srate, 
                            window='hann', 
                            nperseg=winsize, 
                            noverlap=overlap, 
                            nfft=int(winsize * 20),
                            detrend='constant',
                            return_onesided=True, 
                            scaling='density', 
                            axis=-1, 
                            average='mean'
                        )

                        # Filtrage des fréquences jusqu'à 40 Hz
                        f_idx = (freqs <= 40)
                        freqs = freqs[f_idx]
                        chan_psd = chan_psd[:, f_idx]

                        sub_psds.append(chan_psd)
                else: 
                    print(f"Processing subject {subject} with age {subject_age} and session {session} doesn't exist")

    sub_psds = np.array(sub_psds)    
    return freqs, sub_psds


In [27]:
freqs, sub_three_psds=compute_psds('three')
freqs, sub_six_psds=compute_psds('six')
freqs, sub_twelve_psds=compute_psds('twelve')

Processing row: {'subject_id': 'sub-201', 'orig_participant_id': 'LZ', 'session': 'ses-01', 'age': 'three', 'sex': 'M', 'eeg_system': 'EGI'}
Skipping subject sub-201 (EEG system: EGI)
Processing row: {'subject_id': 'sub-201', 'orig_participant_id': 'LZ', 'session': 'ses-02', 'age': 'three', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-201 (EEG system: )
Processing row: {'subject_id': 'sub-201', 'orig_participant_id': 'LZ', 'session': 'ses-03', 'age': 'six', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-201 (EEG system: )
Processing row: {'subject_id': 'sub-201', 'orig_participant_id': 'LZ', 'session': 'ses-04', 'age': 'six', 'sex': 'M', 'eeg_system': 'EGI'}
Skipping subject sub-201 (EEG system: EGI)
Processing row: {'subject_id': 'sub-201', 'orig_participant_id': 'LZ', 'session': 'ses-05', 'age': 'twelve', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-201 (EEG system: )
Processing row: {'subject_id': 'sub-201', 'orig_participant_id': 'LZ', 'session': 'ses-06', 'age': 't

/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)
/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)


Processing row: {'subject_id': 'sub-250', 'orig_participant_id': 'PZ', 'session': 'ses-02', 'age': 'six', 'sex': 'M', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-250', 'orig_participant_id': 'PZ', 'session': 'ses-03', 'age': 'twelve', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-250 (EEG system: )
Processing row: {'subject_id': 'sub-251', 'orig_participant_id': 'AP', 'session': 'ses-01', 'age': 'three', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-251 (EEG system: )
Processing row: {'subject_id': 'sub-251', 'orig_participant_id': 'AP', 'session': 'ses-02', 'age': 'six', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-251 (EEG system: )
Processing row: {'subject_id': 'sub-251', 'orig_participant_id': 'AP', 'session': 'ses-03', 'age': 'twelve', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-251 (EEG system: )
Processing row: {'subject_id': 'sub-252', 'orig_participant_id': 'TE', 'session': 'ses-01', 'age': 'three', 'sex': 'M', 'eeg_system': 'BrainV

/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)
/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)


Processing row: {'subject_id': 'sub-252', 'orig_participant_id': 'TE', 'session': 'ses-02', 'age': 'twelve', 'sex': 'M', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-253', 'orig_participant_id': 'RS', 'session': 'ses-01', 'age': 'three', 'sex': 'F', 'eeg_system': 'BrainVision'}
Processing subject sub-253 with age three (BrainVision)
/home/common/bonaiuto/devmobeta/derivatives/sub-253/ses-01
Processing subject sub-253 with age three and session ses-01


...

Processing row: {'subject_id': 'sub-276', 'orig_participant_id': 'EG', 'session': 'ses-01', 'age': 'three', 'sex': 'F', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-276', 'orig_participant_id': 'EG', 'session': 'ses-02', 'age': 'six', 'sex': 'F', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-276', 'orig_participant_id': 'EG', 'session': 'ses-03', 'age': 'twelve', 'sex': 'F', 'eeg_system': 'BrainVision'}
Processing subject sub-276 with age twelve (BrainVision)
/home/common/bonaiuto/devmobeta/derivatives/sub-276/ses-03
Processing subject sub-276 with age twelve and session ses-03


/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)
/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)


Processing row: {'subject_id': 'sub-277', 'orig_participant_id': 'JA', 'session': 'ses-01', 'age': 'three', 'sex': 'F', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-278', 'orig_participant_id': 'EC', 'session': 'ses-01', 'age': 'three', 'sex': 'M', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-278', 'orig_participant_id': 'EC', 'session': 'ses-02', 'age': 'six', 'sex': 'M', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-278', 'orig_participant_id': 'EC', 'session': 'ses-03', 'age': 'twelve', 'sex': 'M', 'eeg_system': 'BrainVision'}
Processing subject sub-278 with age twelve (BrainVision)
/home/common/bonaiuto/devmobeta/derivatives/sub-278/ses-03
Processing subject sub-278 with age twelve and session ses-03


/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)
/tmp/ipykernel_21808/1875974191.py:34: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab(fname, preload=True)


Processing row: {'subject_id': 'sub-279', 'orig_participant_id': 'LS', 'session': 'ses-01', 'age': 'three', 'sex': 'M', 'eeg_system': ''}
Skipping subject sub-279 (EEG system: )
Processing row: {'subject_id': 'sub-279', 'orig_participant_id': 'LS', 'session': 'ses-02', 'age': 'six', 'sex': 'M', 'eeg_system': 'BrainVision'}
Processing row: {'subject_id': 'sub-280', 'orig_participant_id': 'ADR', 'session': 'ses-01', 'age': 'three', 'sex': 'F', 'eeg_system': ''}
Skipping subject sub-280 (EEG system: )
Processing row: {'subject_id': 'sub-280', 'orig_participant_id': 'ADR', 'session': 'ses-02', 'age': 'six', 'sex': 'F', 'eeg_system': 'BrainVision'}


In [28]:
print(f'N 3m: {sub_three_psds.shape[0]}')
print(f'N 6m: {sub_six_psds.shape[0]}')
print(f'N 12m: {sub_twelve_psds.shape[0]}')

N 3m: 20
N 6m: 21
N 12m: 17


In [ ]:
plt.plot(freqs,np.log10(np.mean(np.mean(sub_three_psds,axis=1),axis=0)),label='3m')
plt.plot(freqs,np.log10(np.mean(np.mean(sub_six_psds,axis=1),axis=0)),label='6m')
plt.plot(freqs,np.log10(np.mean(np.mean(sub_twelve_psds,axis=1),axis=0)),label='12m')
plt.legend()

In [ ]:
def parameterize_psds(freqs, sub_psds):
    n_chans=sub_psds.shape[1]
    fg = FOOOFGroup()
    fgs=fit_fooof_3d(fg, freqs, sub_psds, [.05, 40])
    
    sub_periodic=np.zeros(sub_psds.shape)
    sub_aperiodic=np.zeros(sub_psds.shape)
    for s_idx, fg in enumerate(fgs):
        for chan in range(n_chans):
            if chan in fg.null_inds_:
                sub_aperiodic[s_idx,chan,:]=float('NaN')
                sub_periodic[s_idx,chan,:]=float('NaN')
            else:
                fm = fg.get_fooof(ind=chan, regenerate=True)
                ap_params=fm.get_params('aperiodic_params')
                sub_aperiodic[s_idx,chan,:] = gen_aperiodic(freqs, ap_params)
                sub_periodic[s_idx,chan,:]=np.log10(sub_psds[s_idx,chan,:])-sub_aperiodic[s_idx,chan,:]
                print(sub_periodic.shape)
    return fgs, sub_periodic, sub_aperiodic

In [ ]:
fgs_3m, sub_three_periodic, sub_three_aperiodic=parameterize_psds(freqs, sub_three_psds)
fgs_6m, sub_six_periodic, sub_six_aperiodic=parameterize_psds(freqs, sub_six_psds)
fgs_12m, sub_twelve_periodic, sub_twelve_aperiodic=parameterize_psds(freqs, sub_twelve_psds)

In [ ]:
mean_psd_3m=np.mean(np.mean(sub_three_periodic,axis=1),axis=0)
mean_psd_3m[mean_psd_3m<0]=0
mean_psd_6m=np.mean(np.mean(sub_six_periodic,axis=1),axis=0)
mean_psd_6m[mean_psd_6m<0]=0
mean_psd_12m=np.mean(np.mean(sub_twelve_periodic,axis=1),axis=0)
mean_psd_12m[mean_psd_12m<0]=0

plt.plot(freqs,mean_psd_3m,label='3m')
plt.plot(freqs,mean_psd_6m,label='6m')
plt.plot(freqs,mean_psd_12m,label='12m')
plt.plot(plt.xlim(), [0, 0],'k--')
plt.title('Mean Power Spectral Density (PSD)')
plt.legend()

In [ ]:
all_peaks_3m = []
all_peaks_6m = []
all_peaks_12m = []

# Pour 3 mois
for s_idx, fg in enumerate(fgs_3m):
    for chan in range(len(fg.power_spectra)):  # Assure que chan est dans les limites
        fm = fg.get_fooof(ind=chan, regenerate=True)
        peaks = fm.get_params('peak_params', 'CF')
        if isinstance(peaks, np.ndarray):
            all_peaks_3m.extend(peaks.tolist())  
        else:
            all_peaks_3m.append(peaks)

# Pour 6 mois
for s_idx, fg in enumerate(fgs_6m):
    for chan in range(len(fg.power_spectra)):
        fm = fg.get_fooof(ind=chan, regenerate=True)
        peaks = fm.get_params('peak_params', 'CF')
        if isinstance(peaks, np.ndarray):
            all_peaks_6m.extend(peaks.tolist())
        else:
            all_peaks_6m.append(peaks)

# Pour 12 mois
for s_idx, fg in enumerate(fgs_12m):
    for chan in range(len(fg.power_spectra)):
        fm = fg.get_fooof(ind=chan, regenerate=True)
        peaks = fm.get_params('peak_params', 'CF')
        if isinstance(peaks, np.ndarray):
            all_peaks_12m.extend(peaks.tolist())
        else:
            all_peaks_12m.append(peaks)

# Affichage des histogrammes
plt.hist(all_peaks_3m, 75, alpha=.2, density=True, label='3m')
plt.hist(all_peaks_6m, 75, alpha=.2, density=True, label='6m')
plt.hist(all_peaks_12m, 75, alpha=.2, density=True, label='12m')
plt.legend()
plt.show()


In [ ]:
def define_fois(freqs, mean_psd, age, debug=False):
    mean_resid=mean_psd
    foi_pk_freqs=[]
    foi_pk_vals=[]    
    foi_fwhms=[]
    foi_ranges=[]
    foi_info = {} 
    foi_counter = 1  
    
    orig_pks,properties=scipy.signal.find_peaks(mean_resid)
    while True:
        pks,properties=scipy.signal.find_peaks(mean_resid)

        pk_vals=mean_resid[pks]
        sorted_idx=np.argsort(-pk_vals)
        pk_vals=pk_vals[sorted_idx]
        pks=pks[sorted_idx]

        pk_idx=pks[0]
        pk_val=pk_vals[0]
        pk_freq=freqs[pk_idx]
        
        if pk_val<np.std(mean_resid):
            if debug:
                print('Peak val={}, thresh={}'.format(pk_val,.5*np.std(mean_resid)))
            break
        
        l_idx=np.where(mean_resid[:pk_idx]<=pk_val*.5)[0]
        r_idx=np.where(mean_resid[pk_idx:]<=pk_val*.5)[0]

        if len(l_idx) and len(r_idx):
            l_freq=freqs[l_idx[-1]]
            r_freq=freqs[pk_idx+r_idx[0]]
            r_side=(r_freq-pk_freq)
            l_side=(pk_freq-l_freq)
            fwhm=2*np.min([r_side, l_side])
        elif len(l_idx):
            l_freq=freqs[l_idx[-1]]
            fwhm=2*(pk_freq-l_freq)
        elif len(r_idx):
            r_freq=freqs[pk_idx+r_idx[0]+1]
            fwhm=2*(r_freq-pk_freq)

        l_freq=pk_freq-fwhm*.5
        r_freq=pk_freq+fwhm*.5
        sd=fwhm/(2*np.sqrt(2*np.log(2)))
        A=pk_vals[0]*np.exp(-.5*((freqs-pk_freq)/sd)**2)
        nearest_orig=np.min(np.abs(freqs[np.array(orig_pks)]-pk_freq))
            
        if pk_val>np.std(mean_resid) and ((pk_freq<10 and fwhm>2) or (pk_freq>=10 and fwhm>2.4)):# and nearest_orig<3:
            print('Peak: freq={}, val={}, width={}, range={}-{}'.format(pk_freq, pk_val, fwhm, pk_freq-fwhm*.5, pk_freq+fwhm*.5))
            print('Distance to nearest original peak={}'.format(nearest_orig))
            if debug:
                plt.figure(figsize=(12,4))
                plt.subplot(1,2,1)
                plt.plot(freqs,mean_resid)
                plt.plot(freqs[pks],mean_resid[pks],'ro')
                plt.subplot(1,2,2)
                plt.plot(freqs,mean_resid)
                plt.plot(freqs,A)
            
            foi_pk_freqs.append(pk_freq)
            foi_pk_vals.append(pk_val)
            foi_fwhms.append(fwhm)
            foi_ranges.append([l_freq,r_freq])

            band_name = f'FOI_{foi_counter}'
            foi_info[band_name] = {'frequency_range': [l_freq, r_freq], 'peak_frequency': pk_freq}
            foi_counter += 1
            
        elif debug:
            if not ((pk_freq<10 and fwhm>2) or (pk_freq>=10 and fwhm>2.4)):
                print('Peak: freq={}, fhwm={} to narrow'.format(pk_freq, fwhm))
            if pk_val<=np.std(mean_resid):
                print('Peak: freq={} too low amplitude'.format(pk_freq))
            #if nearest_orig>=3:
            #    print('Peak: freq={} too far from original peaks'.format(pk_freq))

        mean_resid=mean_resid-A
        #mean_resid[mean_resid<0]=0
    sorted_idx=np.argsort(foi_pk_freqs)
    
    foi_pk_freqs=np.array(foi_pk_freqs)
    foi_pk_vals=np.array(foi_pk_vals)
    foi_fwhms=np.array(foi_fwhms)
    foi_ranges=np.array(foi_ranges)
    
    foi_pk_freqs=foi_pk_freqs[sorted_idx]
    foi_pk_vals=foi_pk_vals[sorted_idx]
    foi_fwhms=foi_fwhms[sorted_idx]
    foi_ranges=foi_ranges[sorted_idx,:]

    with open(f'frequency_ranges_{age}_BV_all_channels.json', 'w') as json_file:
        json.dump(foi_info, json_file)
    
    plt.figure()
    plt.plot(freqs,mean_psd, label='Mean PSD')
    for f_idx in range(foi_ranges.shape[0]):
        sd=foi_fwhms[f_idx]/(2*np.sqrt(2*np.log(2)))
        A=foi_pk_vals[f_idx]*np.exp(-.5*((freqs-foi_pk_freqs[f_idx])/sd)**2)
        plt.plot(freqs,A)
        plt.title(f'Peak Detection for {age} months')
    
    return foi_info, foi_pk_freqs,foi_pk_vals,foi_fwhms,foi_ranges

In [ ]:
foi_info_3m, foi_pk_freqs_3m,foi_pk_vals_3m,foi_fwhms_3m,foi_ranges_3m=define_fois(freqs, mean_psd_3m, 'three', debug=False)

In [ ]:
foi_info_6m, foi_pk_freqs_6m,foi_pk_vals_6m,foi_fwhms_6m,foi_ranges_6m=define_fois(freqs, mean_psd_6m, 'six', debug=False)

In [ ]:
foi_info_12m,foi_pk_freqs_12m,foi_pk_vals_12m,foi_fwhms_12m,foi_ranges_12m=define_fois(freqs, mean_psd_12m, 'twelve', debug=False)

In [ ]:
import scipy.io

# Définir le nom du fichier (il sera sauvegardé dans le dossier actuel de Jupyter)
fname = "processed_psd_three_V2.mat"
output_path = fname  

# Sauvegarder les données dans le fichier .mat
scipy.io.savemat(output_path,
                 {
                     'frex': freqs,
                     'psd': sub_three_psds,
                     'periodic': sub_three_periodic,
                     'aperiodic': sub_three_aperiodic,
                     'foi_pk_freqs': foi_pk_freqs_3m,
                     'foi_pk_vals': foi_pk_vals_3m,
                     'foi_fwhms': foi_fwhms_3m,
                     'foi_ranges': foi_ranges_3m
                 })  

In [ ]:
import scipy.io

# Définir le nom du fichier (il sera sauvegardé dans le dossier actuel de Jupyter)
fname = "processed_psd_six_V2.mat"
output_path = fname  

# Sauvegarder les données dans le fichier .mat
scipy.io.savemat(output_path,
                 {
                     'frex': freqs,
                     'psd': sub_six_psds,
                     'periodic': sub_six_periodic,
                     'aperiodic': sub_six_aperiodic,
                     'foi_pk_freqs': foi_pk_freqs_6m,
                     'foi_pk_vals': foi_pk_vals_6m,
                     'foi_fwhms': foi_fwhms_6m,
                     'foi_ranges': foi_ranges_6m
                 })  

In [ ]:
import scipy.io

# Définir le nom du fichier (il sera sauvegardé dans le dossier actuel de Jupyter)
fname = "processed_psd_twelve_V2.mat"
output_path = fname  

# Sauvegarder les données dans le fichier .mat
scipy.io.savemat(output_path,
                 {
                     'frex': freqs,
                     'psd': sub_twelve_psds,
                     'periodic': sub_twelve_periodic,
                     'aperiodic': sub_twelve_aperiodic,
                     'foi_pk_freqs': foi_pk_freqs_12m,
                     'foi_pk_vals': foi_pk_vals_12m,
                     'foi_fwhms': foi_fwhms_12m,
                     'foi_ranges': foi_ranges_12m
                 })  

In [ ]:
foi_ranges_3m

In [ ]:
foi_ranges_6m 

In [ ]:
foi_ranges_12m